## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

In [5]:
##### Your Code Here #####
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from keras.datasets import boston_housing
from sklearn import preprocessing


# fix random seed for reproducibility
np.random.seed(420)

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

In [2]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((404, 13), (404,), (102, 13), (102,))

In [3]:
scaler = preprocessing.StandardScaler().fit(x_train)
scaler.transform(x_train)


array([[-0.27224633, -0.48361547, -0.43576161, ...,  1.14850044,
         0.44807713,  0.8252202 ],
       [-0.40342651,  2.99178419, -1.33391162, ..., -1.71818909,
         0.43190599, -1.32920239],
       [ 0.1249402 , -0.48361547,  1.0283258 , ...,  0.78447637,
         0.22061726, -1.30850006],
       ...,
       [-0.40202987,  0.99079651, -0.7415148 , ..., -0.71712291,
         0.07943894, -0.67776904],
       [-0.17292018, -0.48361547,  1.24588095, ..., -1.71818909,
        -0.98764362,  0.42083466],
       [-0.40422614,  2.04394792, -1.20161456, ..., -1.30866202,
         0.23317118, -1.15392266]])

In [5]:
model = Sequential()
model.add(Dense(15, input_dim=13, activation="relu"))
model.add(Dense(15, activation="relu"))
model.add(Dense(15, activation="relu"))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['mse'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 15)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_3 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 16        
Total params: 706
Trainable params: 706
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, epochs=150)


Instructions for updating:
Use tf.cast instead.
Epoch 1/150


In [ ]:
# evaluate the model
scores = model.evaluate(x_train, y_train)
print(f"{model.metrics_names[1]}: {scores[1]}")

404/404 [==============================] - 0s 338us/step
mean_squared_error: 542.3704078598778


In [ ]:
history = model.fit(x_train, y_train, epochs=150)



Epoch 1/150
404/404 [==============================] - 0s 98us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 2/150
404/404 [==============================] - 0s 100us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 3/150
404/404 [==============================] - 0s 87us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 4/150
404/404 [==============================] - 0s 90us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 5/150
404/404 [==============================] - 0s 89us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 6/150
404/404 [==============================] - 0s 86us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 7/150
404/404 [==============================] - 0s 87us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 8/150
404/404 [==============================] - 0s 84us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 9/150
404/404 [==============================] - 0s 90us/

404/404 [==============================] - 0s 87us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 71/150
404/404 [==============================] - 0s 89us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 72/150
404/404 [==============================] - 0s 85us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 73/150
404/404 [==============================] - 0s 76us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 74/150
404/404 [==============================] - 0s 81us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 75/150
404/404 [==============================] - 0s 80us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 76/150
404/404 [==============================] - 0s 81us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 77/150
404/404 [==============================] - 0s 84us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 78/150
404/404 [==============================] - 0s 72us/step 

404/404 [==============================] - 0s 99us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 140/150
404/404 [==============================] - 0s 93us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 141/150
404/404 [==============================] - 0s 90us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 142/150
404/404 [==============================] - 0s 82us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 143/150
404/404 [==============================] - 0s 82us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 144/150
404/404 [==============================] - 0s 79us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 145/150
404/404 [==============================] - 0s 81us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 146/150
404/404 [==============================] - 0s 76us/step - loss: -341.0881 - mean_squared_error: 542.3704
Epoch 147/150
404/404 [==============================] - 0s 79

In [ ]:
import matplotlib.pyplot as plt

training_loss = history.history['loss']
epoch_count = range(1, len(training_loss) + 1)

plt.plot(epoch_count, training_loss, 'r--')
plt.show();

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

LR = LinearRegression().fit(x_train, y_train)

y_pred = LR.predict(x_train)

mse(y_train, y_pred)

22.004800838348142

###### 

## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- Make sure to have your final layer have as many nodes as the number of classes that you want to predict.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [2]:
##### Your Code Here #####
from keras.datasets import fashion_mnist
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()



/Users/joshsolis/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#global hyperparameters
import keras

batch_size = 64
num_classes = 10
epochs = 50

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train /= 255
x_test /= 255

x_train.shape, x_test.shape

((60000, 784), (10000, 784))

In [6]:
model = Sequential()
model.add(Dense(16, activation = 'relu', input_shape=(784,)))
model.add(Dropout(0.1))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 16)                12560     
_________________________________________________________________
dropout_4 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_6 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                170       
Total para

In [ ]:
history = model.fit(x_train, y_train, epochs=epochs, validation_split=.1)
scores = model.evaluate(x_test,y_test)

Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 27s 506us/step - loss: 0.1549 - acc: 0.9389 - val_loss: 0.0953 - val_acc: 0.9617
Epoch 2/50
54000/54000 [==============================] - 9s 167us/step - loss: 0.1131 - acc: 0.9548 - val_loss: 0.0859 - val_acc: 0.9654
Epoch 3/50
54000/54000 [==============================] - 8s 146us/step - loss: 0.1059 - acc: 0.9580 - val_loss: 0.0872 - val_acc: 0.9659
Epoch 4/50
54000/54000 [==============================] - 8s 149us/step - loss: 0.1010 - acc: 0.9595 - val_loss: 0.0812 - val_acc: 0.9680
Epoch 5/50
54000/54000 [==============================] - 8s 153us/step - loss: 0.0988 - acc: 0.9607 - val_loss: 0.0801 - val_acc: 0.9700
Epoch 6/50
54000/54000 [==============================] - 11s 203us/step - loss: 0.0970 - acc: 0.9615 - val_loss: 0.0806 - val_acc: 0.9683
Epoch 7/50
54000/54000 [==============================] - 14s 265us/step - loss: 0.0954 - acc: 0.9622 - val_loss: 0.0796 

In [ ]:
import matplotlib.pyplot as plt

training_loss = history.history['loss']
epoch_count = range(1, len(training_loss) + 1)

plt.plot(epoch_count, training_loss, 'r--')
plt.show();

## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.